<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/mt5_1500ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Jan 20 18:45:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    57W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install simpletransformers
!pip install torch
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd

data = pd.read_csv('/content/gdrive/MyDrive/dataset1500.csv', usecols=['Thai', 'English'])
data

,English,Thai
0,There are several agricultural residues in the...,มีวัสดุเหลือใช้ทางการเกษตรมากมายในประเทศที่สาม...
1,"At present, biomass is being used in the small...",ปัจจุบันมีการใช้ชีวมวลในการผลิตไฟฟ้าขนาดเล็ก
2,It is an effective way to eliminate agricultur...,เป็นวิธีที่มีประสิทธิภาพในการกำจัดของเสียทางกา...
3,"Recently, there are many incentives from the g...",เมื่อเร็ว ๆ นี้มีแรงจูงใจมากมายจากรัฐบาลสำหรับ...
4,"For examples, these electricity producers are ...",ตัวอย่างเช่น ผู้ผลิตไฟฟ้าเหล่านี้ได้รับอนุญาตใ...
...,...,...
1495,Labor productivity index increases 69.23%.,ดัชนีผลิตภาพแรงงานสูงขึ้น 69.23%
1496,Raw material productivity index increases 3.85%.,ดัชนีผลิตภาพวัตถุดิบเพิ่มสูงขึ้น 3.85%
1497,The total production index increases 13.04%. T...,ดัชนีผลิตภาพรวมเพิ่มขึ้น 13.04% และสามารถกำหนด...
1498,This research presented the simulation of heat...,การวิจัยนี้เสนอการคำนวณเชิงตัวเลขโดยมีระเบียบว...


In [5]:
!rm -rf '/content/train.csv'
!rm -rf '/content/eval.csv'

from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(data, train_size=0.85, shuffle=True)
train_data.to_csv('/content/train.csv', index=False)
eval_data.to_csv('/content/eval.csv', index=False)

In [6]:
pd.read_csv('/content/train.csv')

,English,Thai
0,Due to the low production cost and the cost of...,เนื่องจากต้นทุนการผลิตที่ต่ำและต้นทุนการกำจัดก...
1,"Thus, the power plant makes a profit of 1.48 b...",ทำให้โรงไฟฟ้ามีกำไร 1.48 บาทต่อกิโลวัตต์ชั่วโมง
2,Using floating materials such as plastic float...,การลดอัตราการระเหยจากอ่างเก็บน้ำโดยใช้วัสดุลอย...
3,It may be caused by the tip of the orifice bre...,ให้ค่าการระบายความร้อนในเทอมของเลขนัสเซลต์มากส...
4,Helps resist corrosion (CRAs) and when conside...,ช่วยต้านทานการสึกกร่อน (CRAs) และเมื่อพิจารณาค...
...,...,...
1270,And the COD value of the solution decreased fr...,"และค่าซีโอดีของสารละลายลดลงจาก 28,200 เป็น 27,..."
1271,Complications and solutions to problem found i...,มีการนำเสนอความยุ่งยากซับซ้อนและการแก้ปัญหาที่...
1272,"One is the control, another is the experiment ...",ชุดหนึ่งเป็นชุดควบคุม อีกชุดหนึ่งเป็นชุดทดลองท...
1273,Statistically significant at the 0.05 level.,อย่างมีนัยสำคัญทางสถิติที่ระดับ 0.05


In [7]:
pd.read_csv('/content/eval.csv')

,English,Thai
0,There are five criteria to be used in the deci...,เกณฑ์ที่ใช้ในการตัดสินใจมี 5 เกณฑ์ ได้แก่ ความ...
1,"As well as Net Present Value (NPV) 109,648.03 ...","เช่นเดียวกับมูลค่าปัจจุบันสุทธิ (NPV) 109,648...."
2,Divided into 3 levels of severity Study resul...,แบ่งเป็น 3 ระดับความรุนแรง ผลการศึกษา
3,For most of the sawing timber.,สำหรับกระบวนการเลื่อยไม้ซุงส่วนใหญ่
4,Solar electric vehicle drives using dual-mode ...,รถยนต์ไฟฟ้าพลังงานแสงอาทิตย์ขับเคลื่อนโดยใช้โห...
...,...,...
220,A pin with a low number of convex surfaces and...,โดยสลักที่ผิวด้านนอกมีจำนวนผิวนูนและจำนวนแถวขอ...
221,We use the property of electromagnetic wave fr...,โดยอาศัยสัญญาณคลื่นแม่เหล็กไฟฟ้าความถี่ 22 เฮิ...
222,The field experiment is carried out at Water S...,การทดลองดำเนินการที่ หน่วยผลิตน้ำประปา ส่วนอาค...
223,By measuring the volume all the way around the...,โดยการวัดปริมาตรรอบแผนกและ 8 สายการผลิตรวมถึง ...


In [8]:
train_data = pd.read_csv('/content/train.csv')
eval_data = pd.read_csv('/content/eval.csv')

!rm -rf '/content/train.csv'
!rm -rf '/content/eval.csv'

import csv

with open('/content/train.csv', 'w') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(['prefix', 'input_text', 'target_text'])
  for i in range(len(train_data)):
    writer.writerow(['translate thai to english', train_data['Thai'][i], train_data['English'][i]])
    writer.writerow(['translate english to thai', train_data['English'][i], train_data['Thai'][i]])
with open('/content/eval.csv', 'w') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(['prefix', 'input_text', 'target_text'])
  for i in range(len(eval_data)):
    writer.writerow(['translate thai to english', eval_data['Thai'][i], eval_data['English'][i]])
    writer.writerow(['translate english to thai', eval_data['English'][i], eval_data['Thai'][i]])

pd.read_csv('/content/train.csv')

,prefix,input_text,target_text
0,translate thai to english,เนื่องจากต้นทุนการผลิตที่ต่ำและต้นทุนการกำจัดก...,Due to the low production cost and the cost of...
1,translate english to thai,Due to the low production cost and the cost of...,เนื่องจากต้นทุนการผลิตที่ต่ำและต้นทุนการกำจัดก...
2,translate thai to english,ทำให้โรงไฟฟ้ามีกำไร 1.48 บาทต่อกิโลวัตต์ชั่วโมง,"Thus, the power plant makes a profit of 1.48 b..."
3,translate english to thai,"Thus, the power plant makes a profit of 1.48 b...",ทำให้โรงไฟฟ้ามีกำไร 1.48 บาทต่อกิโลวัตต์ชั่วโมง
4,translate thai to english,การลดอัตราการระเหยจากอ่างเก็บน้ำโดยใช้วัสดุลอย...,Using floating materials such as plastic float...
...,...,...,...
2545,translate english to thai,"One is the control, another is the experiment ...",ชุดหนึ่งเป็นชุดควบคุม อีกชุดหนึ่งเป็นชุดทดลองท...
2546,translate thai to english,อย่างมีนัยสำคัญทางสถิติที่ระดับ 0.05,Statistically significant at the 0.05 level.
2547,translate english to thai,Statistically significant at the 0.05 level.,อย่างมีนัยสำคัญทางสถิติที่ระดับ 0.05
2548,translate thai to english,มีค่าการพยากรณ์ของคุณสมบัติต้านทานแรงอัดดีสุด ...,The predicted compressive strength value from ...


In [9]:
pd.read_csv('/content/eval.csv')

,prefix,input_text,target_text
0,translate thai to english,เกณฑ์ที่ใช้ในการตัดสินใจมี 5 เกณฑ์ ได้แก่ ความ...,There are five criteria to be used in the deci...
1,translate english to thai,There are five criteria to be used in the deci...,เกณฑ์ที่ใช้ในการตัดสินใจมี 5 เกณฑ์ ได้แก่ ความ...
2,translate thai to english,"เช่นเดียวกับมูลค่าปัจจุบันสุทธิ (NPV) 109,648....","As well as Net Present Value (NPV) 109,648.03 ..."
3,translate english to thai,"As well as Net Present Value (NPV) 109,648.03 ...","เช่นเดียวกับมูลค่าปัจจุบันสุทธิ (NPV) 109,648...."
4,translate thai to english,แบ่งเป็น 3 ระดับความรุนแรง ผลการศึกษา,Divided into 3 levels of severity Study resul...
...,...,...,...
445,translate english to thai,The field experiment is carried out at Water S...,การทดลองดำเนินการที่ หน่วยผลิตน้ำประปา ส่วนอาค...
446,translate thai to english,โดยการวัดปริมาตรรอบแผนกและ 8 สายการผลิตรวมถึง ...,By measuring the volume all the way around the...
447,translate english to thai,By measuring the volume all the way around the...,โดยการวัดปริมาตรรอบแผนกและ 8 สายการผลิตรวมถึง ...
448,translate thai to english,มีการจัดการพัฒนาทักษะส่วนบุคคลของพนักงานในการใ...,There is a management to develop personal skil...


In [10]:
import logging
import torch
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.read_csv('/content/train.csv').astype(str)
eval_df = pd.read_csv('/content/eval.csv').astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 4
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Thai-English Translation"

device = torch.cuda.is_available()
model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=device)

In [ ]:
# !wandb login --relogin

In [11]:
!rm -rf '/content/outputs'
!rm -rf '/content/runs'
!rm -rf '/content/wandb'

# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/2550 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: defymiygfl (sigmarai). Use `wandb login --relogin` to force relogin


Running Epoch 0 of 4:   0%|          | 0/319 [00:00<?, ?it/s]

...

Running Epoch 2 of 4:   0%|          | 0/319 [00:00<?, ?it/s]

  0%|          | 0/450 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/319 [00:00<?, ?it/s]

  0%|          | 0/450 [00:00<?, ?it/s]

(1276,
 {'global_step': [319, 638, 957, 1276],
  'eval_loss': [2.9572517997340153,
   2.486713323676795,
   2.326535149624473,
   2.3611450571762886],
  'train_loss': [4.24097204208374,
   2.7421443462371826,
   2.252577543258667,
   1.9440813064575195]})

In [12]:
# Evaluate the model
result = model.eval_model(eval_df)

  0%|          | 0/450 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/57 [00:00<?, ?it/s]

In [13]:
import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 96
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "/content/outputs", args=model_args)

In [14]:
valid = pd.read_csv("/content/eval.csv").astype(str)

thai_truth = [valid.loc[valid["prefix"] == "translate english to thai"]["target_text"].tolist()]
to_thai = valid.loc[valid["prefix"] == "translate english to thai"]["input_text"].tolist()

english_truth = [valid.loc[valid["prefix"] == "translate thai to english"]["target_text"].tolist()]
to_english = valid.loc[valid["prefix"] == "translate thai to english"]["input_text"].tolist()

In [15]:
english_preds = model.predict(to_english)
then_bleu = sacrebleu.corpus_bleu(english_preds, english_truth)
print("Thai to English: ", then_bleu.score)

thai_preds = model.predict(to_thai)
enth_bleu = sacrebleu.corpus_bleu(thai_preds, thai_truth)
print("English to Thai: ", enth_bleu.score)

Generating outputs:   0%|          | 0/29 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/225 [00:00<?, ?it/s]

Thai to English:  13.477188403479017


Generating outputs:   0%|          | 0/29 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/225 [00:00<?, ?it/s]

English to Thai:  8.296490766200813


In [23]:
from simpletransformers.t5 import T5Model

model = T5Model("mt5", "/content/outputs")
to_predict = ['โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ',
              'และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก',
              'โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน',
              'ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน',
              'และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน',
              'แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)',
              'ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน',
              'และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์',
              'ในขั้นตอนการเลือกแบบจําลองการเรียนรู้เชิงลึกเพื่อนําไปประมวลผลบนเครื่อง Jetson Nano']
# to_predict = to_predict[0].split()
predictions = model.predict(to_predict)
for i,info in enumerate(predictions):
  print(to_predict[i] + '\n' + info + '\n')

Generating outputs:   0%|          | 0/2 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/9 [00:00<?, ?it/s]

โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ
This research is a development of a classification system.

และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก
And analyze the vegetable species from the vegetable leaves with detailed

โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน
The objective of this research was to easily analyze the image from the

ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน
The objective of this research was to collect the images of pineapple from the greenhouse

และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน
And created the classification of the defect of the gong.

แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)
Once the data were used to easily modify the physical properties in the image

ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน
The researcher can detect the ผิดปกติ of the mushroom leaves.

และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์
And when the system de